In [1]:
import numpy as np
import torch
import torch.nn as nn

print(torch.__version__)

1.0.1.post2


앞서 연습한 Embedding layer 를 다시 만듭니다.

In [2]:
n_vocabs = 5
embed_dim = 3

embed_np = np.random.random_sample((n_vocabs, embed_dim))

embed = nn.Embedding(
    num_embeddings = n_vocabs,
    embedding_dim = embed_dim
)

embed.weight.data.copy_(torch.from_numpy(embed_np))

tensor([[0.5266, 0.2686, 0.8542],
        [0.0038, 0.7054, 0.4519],
        [0.2445, 0.2388, 0.7067],
        [0.1246, 0.1595, 0.1162],
        [0.7820, 0.4416, 0.5663]])

## Apply convolution

Embedding layer 를 다루는 방법을 익혔으니, Convolution layer 를 다루는 법도 연습합니다.

torch.nn.Conv2d 는 세 가지 argument 를 반드시 확인해야 합니다.

    in_channels : 입력되는 데이터의 channel 개수
    out_channels : 출력되는 결과의 channel 개수 (filter 의 개수)
    kernel_size : Convolution filter 의 크기. 

우리는 word - level CNN 모델을 위한 convolution layer 를 만들어봅니다. Sentence image 의 channel 의 개수가 1 개 이기 때문에 in channels 는 1 개 입니다. out channels = 2 로 지정하였기 때문에 같은 크기의 2 개의 필터가 만들어집니다.

kernel_size 는 (2, embedding dimension) 으로 설정하였습니다. Bigram word vector 를 확인하는 filter 가 됩니다.

In [3]:
conv = nn.Conv2d(
    in_channels=1,
    out_channels=2,
    kernel_size=(2, embed_dim),
    bias=False
)

torch.nn.Conv2d 를 print 하면 현재 만들어진 Convolution layer 의 구조가 출력됩니다. 아래에서 볼 수 있듯이 우리는 row, column 방향의 stride 도 조절할 수 있습니다. Default 는 모두 한칸씩 이동합니다. 우리가 kernel_size 의 column size 를 word vector dimension 으로 만들었기 때문에 열을 따라서 stride 는 일어나지 않습니다.

In [4]:
nn.Conv2d(1, 2, kernel_size=(2, embed_dim), stride=(1, 1))

Conv2d(1, 2, kernel_size=(2, 3), stride=(1, 1))

kernel_size 를 int 로 입력하면 square 형태의 kernel 이 만들어집니다.

In [5]:
nn.Conv2d(1, 2, kernel_size=2, stride=(1, 1))

Conv2d(1, 2, kernel_size=(2, 2), stride=(1, 1))

Convolution filter weight 는 torch.nn.Conv2d.weight 에 저장되어 있습니다. 앞서 out_channels = 2 로 만들었기 때문에 2 개의 filter 가 만들어져 있습니다. weight 는 아래와 같은 구조입니다. 

(out_channels, in_channels, kernel_size[0], kernel_size[1])

In [6]:
print(conv.weight.size())

conv.weight

torch.Size([2, 1, 2, 3])


Parameter containing:
tensor([[[[ 0.3424, -0.1723,  0.3093],
          [-0.3682, -0.0827, -0.0742]]],


        [[[ 0.3427, -0.0415,  0.3559],
          [-0.2641,  0.2919,  0.1172]]]], requires_grad=True)

Convolution layer 에 x 를 입력하기 위해서는 resize 를 통하여 (batch, in_channel, sentence length, word dimension) 으로 그 형식을 변환합니다. 

In [7]:
sent = torch.LongTensor([0, 1, 2, 3, 4])
x = embed(sent)
x

tensor([[0.5266, 0.2686, 0.8542],
        [0.0038, 0.7054, 0.4519],
        [0.2445, 0.2388, 0.7067],
        [0.1246, 0.1595, 0.1162],
        [0.7820, 0.4416, 0.5663]], grad_fn=<EmbeddingBackward>)

Convolution layer 에 적용된 결과입니다. 길이가 5 인 sentence image 에 대하여 bi-gram 이 적용되었기 때문에 [(0, 1), (1, 2), (2, 3), (3, 4)] 의 dot product 값이 두 개 출력됩니다.

In [8]:
# (batch, in channel (num filters), sent_len, embedding_dim)
x = x.view(1, 1, x.size()[0], x.size()[1])
print(x.size())

conv(x)

torch.Size([1, 1, 5, 3])


tensor([[[[ 0.3050],
          [-0.1426],
          [ 0.1935],
          [-0.3153]],

         [[ 0.7313],
          [ 0.2209],
          [ 0.3527],
          [ 0.0662]]]], grad_fn=<MkldnnConvolutionBackward>)

forward 함수의 결과는 (batch, out channel, sent_len, 1) 입니다.

In [9]:
conv(x).size()

torch.Size([1, 2, 4, 1])

앞의 2 개의 단어의 embedding vector 와 첫번째 convolution filter 의 weight 입니다.

In [10]:
print(x[0,0,:2])
print(conv.weight[0,0])

tensor([[0.5266, 0.2686, 0.8542],
        [0.0038, 0.7054, 0.4519]], grad_fn=<SliceBackward>)
tensor([[ 0.3424, -0.1723,  0.3093],
        [-0.3682, -0.0827, -0.0742]], grad_fn=<SelectBackward>)


이들의 element-wise product 의 값이 conv(x)[0,0] 과 같습니다.

In [11]:
torch.dot(
    conv.weight[0,0].view(-1),
    x[0,0,:2].view(-1)
)

tensor(0.3050, grad_fn=<DotBackward>)

In [12]:
conv(x).size()

torch.Size([1, 2, 4, 1])

In [13]:
conv(x)[0,0,0]

tensor([0.3050], grad_fn=<SelectBackward>)

## Pooling

Max pooling 은 torch.nn.functional 을 이용할 수 있습니다. squeeze 를 이용하여 conv(x) 의 size 를 변환합니다. pooling 함수는 어떤 axis 를 기준으로 pooling 할 것인지 kernel_size 로 지정합니다. 지정해야 합니다

```
kernel_size – the size of the window to take a max over
```

In [14]:
out = conv(x).squeeze(dim=3)
out.size()

torch.Size([1, 2, 4])

max_pool1d 의 결과 각 filter 별로 가장 큰 값이 출력됩니다.

In [15]:
import torch.nn.functional as F

out = conv(x).squeeze(dim=3)
print(out)

F.max_pool1d(out, kernel_size=out.size()[2])

tensor([[[ 0.3050, -0.1426,  0.1935, -0.3153],
         [ 0.7313,  0.2209,  0.3527,  0.0662]]], grad_fn=<SqueezeBackward1>)


tensor([[[0.3050],
         [0.7313]]], grad_fn=<SqueezeBackward1>)

max_pool1d_with_indices 를 이용하면 어떤 위치의 값이 가장 큰지 index 도 함께 return 됩니다.

In [16]:
pool, idx = F.max_pool1d_with_indices(out, kernel_size=out.size()[2])

print(pool)
print(idx.squeeze())

tensor([[[0.3050],
         [0.7313]]], grad_fn=<SqueezeBackward1>)
tensor([0, 0])
